In [ ]:
# 指定したフォルダに関して、ファイル名先頭に記載された日時情報を読み取り、動画の生成日時情報を書き換える処理を行っている。
# 処理が成功した場合には、create_date_changedというフォルダに書き換えられたファイルが格納され、元のファイルは先頭に"old_"の文字列が追加される。

import os 
import datetime
import subprocess
import re
from datetime import datetime, timedelta

ffmpeg_path = "C:\\Users\\taqsa\\ffmpeg-2024-05-02-git-71669f2ad5-essentials_build\\bin\\ffmpeg.exe"

def set_creation_time(mp4_file, new_c_time, ffmpeg=ffmpeg_path):
    new_c_time = new_c_time - timedelta(hours=9)
    old_file = os.path.join(os.path.dirname(mp4_file), 'old_' + os.path.basename(mp4_file))
    success = False
    try:
        os.rename(mp4_file, old_file)
        cmd = [ffmpeg, '-i', old_file, '-c', 'copy']
        cmd.extend(['-metadata', f"creation_time={new_c_time.isoformat()}Z"])
        cmd.append(mp4_file)
        ret = subprocess.run(cmd)
        if ret.returncode == 0:
            success = True
    finally:
        if not success:
            print(f'Rename back {old_file} to {mp4_file}')
            os.rename(old_file, mp4_file)


import os
from datetime import datetime, timedelta

def extract_datetime_from_filepath(filepath):
    # ファイル名部分を取得
    filename = os.path.basename(filepath)
    try:
        datetime_str = filename[:16]  # ファイル名の先頭から16文字を取得
        return datetime.strptime(datetime_str, "%Y-%m-%d-%H-%M")
    except ValueError:
        return None

def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        datetime_info = extract_datetime_from_filepath(filepath)
        if datetime_info:
            set_creation_time(filepath, datetime_info)
            # 新しいフォルダを作成してファイルを移動
            new_folder = os.path.join(folder_path, 'create_date_changed')
            os.makedirs(new_folder, exist_ok=True)
            new_filepath = os.path.join(new_folder, os.path.basename(filepath))
            os.rename(filepath, new_filepath)
        else:
            print(f"日時情報が見つかりませんでした: {filepath}")

def main():
    target_folder = input("指定したフォルダのパスをここに入力: ")
    process_folder(target_folder)

if __name__ == "__main__":
    main()


In [ ]:
# 個別のファイルに対応するため、ファイルのパスと日時をインプットするバージョン　動作する。　

import os 
import datetime
import subprocess
import re
import os
from datetime import datetime, timedelta

ffmpeg_path = "C:\\Users\\taqsa\\ffmpeg-2024-05-02-git-71669f2ad5-essentials_build\\bin\\ffmpeg.exe"

def set_creation_time(mp4_file, new_c_time, ffmpeg=ffmpeg_path):
    new_c_time = new_c_time - timedelta(hours=9)   #時差を考慮　入力するdatetimeオブジェクトが標準時なので、日本時間から9時間引いておく必要
    old_file = os.path.join(os.path.dirname(mp4_file),
                            'old_' + os.path.basename(mp4_file))
    success = False
    try:
        os.rename(mp4_file, old_file)
        cmd = [ffmpeg, '-i', old_file, '-c', 'copy']
        cmd.extend(['-metadata', f"creation_time={new_c_time.isoformat()}Z"])
        cmd.append(mp4_file)
        ret = subprocess.run(cmd)
        if ret.returncode == 0:
            success = True
    finally:
        if not success:
            print(f'Rename back {old_file} to {mp4_file}')
            os.rename(old_file, mp4_file)

def get_creation_time(mp4_file, ffmpeg=ffmpeg_path):
    cmd = [ffmpeg, '-i', mp4_file]
    ret = subprocess.run(cmd, capture_output=True, encoding='utf8')
    for line in ret.stderr.splitlines():
        if m:= re.match(r'\s*creation_time\s*:\s*(\S+)Z', line):
            return datetime.datetime.fromisoformat(m.group(1))
    return None


import os
from datetime import datetime, timedelta

def extract_datetime_from_filepath(filepath):
    # ファイル名部分を取得
    filename = os.path.basename(filepath)
    try:
        datetime_str = filename[:16]  # ファイル名の先頭から16文字を取得
        return datetime.strptime(datetime_str, "%Y-%m-%d-%H-%M")
    except ValueError:
        return None

# 使用例
filepath = input("生成日時を変更したいファイルのパス:") #"C:\\Users\\taqsa\\Documents\\Temp 動画編集用\\サイスポ\\20240428サイクルスポーツセンター.mp4"
result = extract_datetime_from_filepath(input("変更したい生成日時(2024-10-23-12-00の形で記載):")) #本来extract_datetime_from_filepathの引数はパスだが、生成日時の文字列を入力しても問題なく動くので、そのまま。
print(result)
set_creation_time(filepath, result)
